XGBoost model based on embeddings from NN model

In [22]:
from google.cloud import storage
from io import BytesIO

client = storage.Client()
storage_client = storage.Client(project = 'irkml1')
bucket = storage_client.get_bucket("aindstorm_bucket")
blob1 = storage.blob.Blob("df_train_emb.csv",bucket)
content1 = blob1.download_as_string()

In [41]:
blob2 = storage.blob.Blob("sab_emb.csv",bucket)
content2 = blob2.download_as_string()

In [4]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn import preprocessing
from sklearn.datasets import load_iris

In [23]:
df_train = pd.read_csv(BytesIO(content1))

In [24]:
le = preprocessing.LabelEncoder()
le.fit(df_train['service_title'])
df_train['service_title'] = le.transform(df_train['service_title'])

In [25]:
scaler = StandardScaler()
scaler.fit(df_train[['age']])
df_train[['age']] = scaler.transform(df_train[['age']])

In [26]:
X = df_train.drop(['service_title'], axis=1)
y = df_train['service_title']

In [27]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [28]:
xgb_model = xgb.XGBClassifier(objective="multi:softmax", eval_metric=['mlogloss'], n_jobs=-1)

In [29]:
xgb_model.fit(X_train, y_train, early_stopping_rounds=5, eval_set=[(X_test, y_test)])

[0]	validation_0-mlogloss:3.88023
Will train until validation_0-mlogloss hasn't improved in 5 rounds.
[1]	validation_0-mlogloss:11.88905
[2]	validation_0-mlogloss:10.16115
[3]	validation_0-mlogloss:10.83159
[4]	validation_0-mlogloss:11.36731
[5]	validation_0-mlogloss:12.19505
Stopping. Best iteration:
[0]	validation_0-mlogloss:3.88023



XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, eval_metric=['mlogloss'],
              gamma=0, gpu_id=-1, importance_type='gain',
              interaction_constraints='', learning_rate=0.300000012,
              max_delta_step=0, max_depth=6, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=100, n_jobs=-1,
              num_parallel_tree=1, objective='multi:softprob', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=None, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [31]:
accuracy_score(y_test, xgb_model.predict(X_test))

0.4083945119051761

In [33]:
y_pred = xgb_model.predict(X_test)

In [35]:
df_pred = pd.DataFrame()
df_pred['y_pred'] = y_pred

In [38]:
df_pred['y_pred'] = le.inverse_transform(y_pred)

In [40]:
df_pred['y_pred'].value_counts(normalize=True)[:30]

603     0.123065
134     0.105494
907     0.068961
98      0.063997
805     0.050149
4       0.038864
207     0.037937
901     0.037067
1205    0.033367
234     0.031917
412     0.027162
1169    0.024843
604     0.022767
155     0.020609
651     0.020354
1259    0.018823
1020    0.016318
949     0.014845
692     0.014231
178     0.011946
870     0.011053
263     0.010879
1295    0.010821
1220    0.010252
170     0.009638
826     0.006854
573     0.006472
339     0.006042
278     0.005335
611     0.005219
Name: y_pred, dtype: float64

In [42]:
sab_emb = pd.read_csv(BytesIO(content2))

In [45]:
sab_emb[['age']] = scaler.transform(sab_emb[['age']])

In [46]:
y_pred = xgb_model.predict(sab_emb)

In [49]:
sab = pd.read_csv('sample_submission_ld.csv')

In [52]:
sab['service_title'] = le.inverse_transform(y_pred)

In [55]:
sab['service_title'].value_counts(normalize=True)

603     0.140131
134     0.105999
98      0.071416
907     0.067973
207     0.062753
          ...   
1275    0.000023
310     0.000023
1129    0.000023
628     0.000023
906     0.000023
Name: service_title, Length: 137, dtype: float64

In [57]:
sab.to_csv('xgb_sub_0.csv', index=False)